In [1]:
import sqlite3
import pandas as pd

DB_PATH = "AAPL_news_v2.db"

# Connexion et chargement de la table principale
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql_query("""
    SELECT id, ticker, published_utc, title, description
    FROM news_articles
    WHERE ticker = 'AAPL'
""", conn)

conn.close()

print("Nb de lignes:", len(df))
df.head()

Nb de lignes: 7480


,id,ticker,published_utc,title,description
0,QILeG_MOlE81CMvvZx_J9MiU4rtfV1i7o8ZtV1VQk1Q,AAPL,2020-06-22T13:15:00Z,"Stocks Slip on Apple News, but Stay Positive f...","Stocks Slip on Apple News, but Stay Positive f..."
1,DKI2nRnVycLp9YEqeKXK0ZYI9udBuOk2V8TezeK_DxI,AAPL,2020-06-23T13:15:00Z,NASDAQ Ends at Record Close Again with Some He...,NASDAQ Ends at Record Close Again with Some He...
2,9wIo--KHVcGi6_UayArHT_hS2sHpoukRTSh04aEraRg,AAPL,2021-01-28T14:15:00Z,Market Plunges More Than 2% While Waiting for ...,Market Plunges More Than 2% While Waiting for ...
3,Z25eTh3Z7Gw_Tm467TlyWkn_hCKRo4fveV0W1J8Yyfw,AAPL,2021-04-20T16:01:10Z,Why Apple Could Lay An Egg,Apple's extended valuation is counting on low ...
4,ubTmRRT-ailyQeTqdALfmjU-ACdAQlxBsRS1FUMb8bc,AAPL,2021-04-22T13:17:51Z,Apple: A New Era For The PC,Apple's 2021 iMac is the first to be built aro...


In [2]:
# Copie de travail
df_clean = df.copy()

# Nettoyage texte
df_clean["title"] = df_clean["title"].fillna("")
df_clean["description"] = df_clean["description"].fillna("")

# Conversion date + extraction jour
df_clean["published_utc"] = pd.to_datetime(df_clean["published_utc"])
df_clean["date"] = df_clean["published_utc"].dt.date

# Texte pour FinBERT = titre + description
df_clean["text_for_sentiment"] = (
    df_clean["title"] + ". " + df_clean["description"]
).str.strip()

# On ne garde que ce qui a du texte
df_clean = df_clean[df_clean["text_for_sentiment"] != ""].reset_index(drop=True)

df_clean[["id", "date", "title", "text_for_sentiment"]].head()

,id,date,title,text_for_sentiment
0,QILeG_MOlE81CMvvZx_J9MiU4rtfV1i7o8ZtV1VQk1Q,2020-06-22,"Stocks Slip on Apple News, but Stay Positive f...","Stocks Slip on Apple News, but Stay Positive f..."
1,DKI2nRnVycLp9YEqeKXK0ZYI9udBuOk2V8TezeK_DxI,2020-06-23,NASDAQ Ends at Record Close Again with Some He...,NASDAQ Ends at Record Close Again with Some He...
2,9wIo--KHVcGi6_UayArHT_hS2sHpoukRTSh04aEraRg,2021-01-28,Market Plunges More Than 2% While Waiting for ...,Market Plunges More Than 2% While Waiting for ...
3,Z25eTh3Z7Gw_Tm467TlyWkn_hCKRo4fveV0W1J8Yyfw,2021-04-20,Why Apple Could Lay An Egg,Why Apple Could Lay An Egg. Apple's extended v...
4,ubTmRRT-ailyQeTqdALfmjU-ACdAQlxBsRS1FUMb8bc,2021-04-22,Apple: A New Era For The PC,Apple: A New Era For The PC. Apple's 2021 iMac...


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

MODEL_NAME = "yiyanghkust/finbert-tone"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

finbert = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=256,
    # device=0  # décommente si tu as un GPU
)

# Application de FinBERT en batch
texts = df_clean["text_for_sentiment"].tolist()
batch_size = 32

sentiments = []
scores = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    results = finbert(batch_texts)
    for res in results:
        label = res["label"].lower().strip()
        # Normalisation des labels
        if label.startswith("pos"):
            label = "positive"
        elif label.startswith("neg"):
            label = "negative"
        elif label.startswith("neu"):
            label = "neutral"
        sentiments.append(label)
        scores.append(float(res["score"]))

df_clean["finbert_sentiment"] = sentiments
df_clean["finbert_score"] = scores

df_clean[["date", "title", "finbert_sentiment", "finbert_score"]].head()

Device set to use mps:0


,date,title,finbert_sentiment,finbert_score
0,2020-06-22,"Stocks Slip on Apple News, but Stay Positive f...",positive,0.559658
1,2020-06-23,NASDAQ Ends at Record Close Again with Some He...,positive,0.996689
2,2021-01-28,Market Plunges More Than 2% While Waiting for ...,negative,0.996833
3,2021-04-20,Why Apple Could Lay An Egg,negative,0.996980
4,2021-04-22,Apple: A New Era For The PC,positive,0.581030


In [5]:
import sqlite3
import pandas as pd

# 1) Mapper le sentiment en nombre
sentiment_map = {"positive": 1, "negative": -1, "neutral": 0}

df_daily = df_clean.copy()
df_daily["sentiment_num"] = df_daily["finbert_sentiment"].map(sentiment_map)

# 2) Additionner par date
df_daily = (
    df_daily
    .groupby("date", as_index=False)["sentiment_num"]
    .sum()
    .rename(columns={"sentiment_num": "daily_sentiment"})
)

print("Après groupby (dates présentes uniquement) :")
print(df_daily.head())

# 🔁 3) Compléter toutes les dates de 2020-01-01 à 2025-12-09 avec sentiment 0 si manquant

# S'assurer que la colonne date est bien en datetime
df_daily["date"] = pd.to_datetime(df_daily["date"])

# Plage complète de dates
full_range = pd.date_range(start="2020-01-01", end="2025-12-09", freq="D")

# Reindexer sur la plage complète
df_full = (
    df_daily
    .set_index("date")
    .reindex(full_range)
    .rename_axis("date")
    .reset_index()
)

# Remplir les sentiments manquants par 0 et caster en int
df_full["daily_sentiment"] = df_full["daily_sentiment"].fillna(0).astype(int)

# Mettre la date au format string "YYYY-MM-DD" pour SQLite
df_full["date"] = df_full["date"].dt.strftime("%Y-%m-%d")

print("Après complétion des dates :")
print(df_full.head())
print(df_full.tail())
print("Nb total de lignes (doit être nb de jours entre 2020-01-01 et 2025-12-09) :", len(df_full))

# 4) Sauvegarder dans un nouveau fichier SQLite
DB_PATH = "AAPL_daily_sentiment_v2.db"
conn = sqlite3.connect(DB_PATH)

# On crée / remplace la table "daily_sentiment"
df_full.to_sql("daily_sentiment", conn, if_exists="replace", index=False)

# Vérification rapide
check = pd.read_sql_query("SELECT * FROM daily_sentiment ORDER BY date LIMIT 5", conn)
check_tail = pd.read_sql_query("SELECT * FROM daily_sentiment ORDER BY date DESC LIMIT 5", conn)
conn.close()

print("Aperçu début dans AAPL_daily_sentiment_v2.db :")
print(check)
print("Aperçu fin dans AAPL_daily_sentiment_v2.db :")
print(check_tail)

Après groupby (dates présentes uniquement) :
         date  daily_sentiment
0  2020-06-22                1
1  2020-06-23                1
2  2021-01-28               -1
3  2021-04-20               -1
4  2021-04-22                2
Après complétion des dates :
         date  daily_sentiment
0  2020-01-01                0
1  2020-01-02                0
2  2020-01-03                0
3  2020-01-04                0
4  2020-01-05                0
            date  daily_sentiment
2165  2025-12-05                1
2166  2025-12-06                4
2167  2025-12-07                0
2168  2025-12-08                1
2169  2025-12-09                0
Nb total de lignes (doit être nb de jours entre 2020-01-01 et 2025-12-09) : 2170
Aperçu début dans AAPL_daily_sentiment_v2.db :
         date  daily_sentiment
0  2020-01-01                0
1  2020-01-02                0
2  2020-01-03                0
3  2020-01-04                0
4  2020-01-05                0
Aperçu fin dans AAPL_daily_sentiment